## Compare results for multiple results

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
# sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
# from modules_img_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [5]:
# ## Histogram modules

def f_batch_histogram(img_arr,bins,norm,hist_range,spread_type='sdev'):
    ''' Compute histogram statistics for a batch of images'''

    ## Extracting the range. This is important to ensure that the different histograms are compared correctly
    if hist_range==None : ulim,llim=np.max(img_arr),np.min(img_arr)
    else: ulim,llim=hist_range[1],hist_range[0]
#         print(ulim,llim)
    ### array of histogram of each image
    hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important
    hist=np.stack(hist_arr[:,0]) # First element is histogram array
#         print(hist.shape)
    bin_list=np.stack(hist_arr[:,1]) # Second element is bin value 
    ### Compute statistics over histograms of individual images
    
    
    if spread_type=='stderr':
        mean,err=np.mean(hist,axis=0),np.std(hist,axis=0)/np.sqrt(hist.shape[0])
    elif spread_type=='sdev':
        mean,err=np.mean(hist,axis=0),np.std(hist,axis=0)

    bin_edges=bin_list[0]
    centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    return mean,err,centers


def f_pixel_intensity(img_arr,bins=25,label='validation',mode='avg',normalize=False,log_scale=True,plot=True, hist_range=None):
    '''
    Module to compute and plot histogram for pixel intensity of images
    Has 2 modes : simple and avg
        simple mode: No errors. Just flatten the input image array and compute histogram of full data
        avg mode(Default) : 
            - Compute histogram for each image in the image array
            - Compute errors across each histogram 
    '''
    
    norm=normalize # Whether to normalize the histogram
    
    if plot: 
        plt.figure()
        plt.xlabel('Pixel value')
        plt.ylabel('Counts')
        plt.title('Pixel Intensity Histogram')

        if log_scale: plt.yscale('log')
    
    if mode=='simple':
        hist, bin_edges = np.histogram(img_arr.flatten(), bins=bins, density=norm, range=hist_range)
        centers = (bin_edges[:-1] + bin_edges[1:]) / 2

        if plot: plt.errorbar(centers, hist, fmt='o-', label=label)
        return hist,None
    
    elif mode=='avg': 
        ### Compute histogram for each image. 
        mean,err,centers=f_batch_histogram(img_arr,bins,norm,hist_range)

        if plot: plt.errorbar(centers,mean,yerr=err,fmt='o-',label=label)  
        return mean,err
    
def f_compare_pixel_intensity(img_lst,label_lst=['img1','img2'],bkgnd_arr=[],log_scale=True, normalize=True, mode='avg',bins=25, hist_range=None):
    '''
    Module to compute and plot histogram for pixel intensity of images
    Has 2 modes : simple and avg
    simple mode: No errors. Just flatten the input image array and compute histogram of full data
    avg mode(Default) : 
        - Compute histogram for each image in the image array
        - Compute errors across each histogram 
        
    bkgnd_arr : histogram of this array is plotting with +/- sigma band
    '''
    
    norm=normalize # Whether to normalize the histogram
    
    plt.figure()
    
    ## Plot background distribution
    if len(bkgnd_arr):
        if mode=='simple':
            hist, bin_edges = np.histogram(bkgnd_arr.flatten(), bins=bins, density=norm, range=hist_range)
            centers = (bin_edges[:-1] + bin_edges[1:]) / 2
            plt.errorbar(centers, hist, color='k',marker='*',linestyle=':', label='bkgnd')

        elif mode=='avg':
            ### Compute histogram for each image. 
            mean,err,centers=f_batch_histogram(bkgnd_arr,bins,norm,hist_range)
            plt.plot(centers,mean,linestyle=':',color='k',label='bkgnd')
            plt.fill_between(centers, mean - err, mean + err, color='k', alpha=0.4)
    
    ### Plot the rest of the datasets
    for img,label,mrkr in zip(img_lst,label_lst,itertools.cycle('>^*sDHPdpx_')):     
        if mode=='simple':
            hist, bin_edges = np.histogram(img.flatten(), bins=bins, density=norm, range=hist_range)
            centers = (bin_edges[:-1] + bin_edges[1:]) / 2
            plt.errorbar(centers, hist, fmt=mrkr+'-', label=label)

        elif mode=='avg':
            ### Compute histogram for each image. 
            mean,err,centers=f_batch_histogram(img,bins,norm,hist_range)
#             print('Centers',centers)
            plt.errorbar(centers,mean,yerr=err,fmt=mrkr+'-',label=label)

    if log_scale: 
        plt.yscale('log')
        plt.xscale('symlog',linthreshx=50)

    plt.legend()
    plt.xlabel('Pixel value')
    plt.ylabel('Counts')
    plt.title('Pixel Intensity Histogram')

In [6]:
## numpy code
def f_radial_profile_3d(data, center=(None,None)):
    ''' Module to compute radial profile of a 2D image '''
    
    z, y, x = np.indices((data.shape)) # Get a grid of x and y values
    
    center=[]
    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0, (z.max()-z.min())/2.0]) # compute centers
        
    # get radial values of every pair of points
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2+ + (z - center[2])**2)
    r = r.astype(np.int)
    
    # Compute histogram of r values
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel()) 
    radialprofile = tbin / nr
    
    return radialprofile[1:-1]

def f_compute_spectrum_3d(arr):
    '''
    compute spectrum for a 3D image
    '''
#     GLOBAL_MEAN=1.0
#     arr=((arr - GLOBAL_MEAN)/GLOBAL_MEAN)
    y1=np.fft.fftn(arr)
    y1=np.fft.fftshift(y1)
#     print(y1.shape)
    y2=abs(y1)**2
    z1=f_radial_profile_3d(y2)
    return(z1)
   
def f_batch_spectrum_3d(arr):
    batch_pk=np.array([f_compute_spectrum_3d(i) for i in arr])
    return batch_pk

### Code ###
def f_image_spectrum_3d(x,num_channels):
    '''
    Compute spectrum when image has a channel index
    Data has to be in the form (batch,channel,x,y)
    '''
    mean=[[] for i in range(num_channels)]    
    sdev=[[] for i in range(num_channels)]    

    for i in range(num_channels):
        arr=x[:,i,:,:,:]
#         print(i,arr.shape)
        batch_pk=f_batch_spectrum_3d(arr)
#         print(batch_pk)
        mean[i]=np.mean(batch_pk,axis=0)
        sdev[i]=np.var(batch_pk,axis=0)
    mean=np.array(mean)
    sdev=np.array(sdev)
    return mean,sdev


def f_plot_spectrum_3d(img_arr,plot=False,label='input',log_scale=True,spread_type='sdev'):
    '''
    Module to compute Average of the 1D spectrum for a batch of 3d images
    '''
    num = img_arr.shape[0]
    Pk = f_batch_spectrum_3d(img_arr)

    if spread_type=='stderr':
        mean,std = np.mean(Pk, axis=0),np.std(Pk, axis=0)/np.sqrt(Pk.shape[0])
    elif spread_type=='sdev':
        mean,std = np.mean(Pk, axis=0),np.std(Pk, axis=0)
    
    k=np.arange(len(mean))
    
    if plot: 
        plt.figure()
        plt.plot(k, mean, 'k:')
        plt.plot(k, mean + std, 'k-',label=label)
        plt.plot(k, mean - std, 'k-')
    #     plt.xscale('log')
        if log_scale: plt.yscale('log')
        plt.ylabel(r'$P(k)$')
        plt.xlabel(r'$k$')
        plt.title('Power Spectrum')
        plt.legend()

    return mean,std

In [7]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None,spread_type='stderr')
    ### Compute spectrum for row
    # spec,spec_sdev=f_plot_spectrum_3d(sample,plot=False,spread_type='stderr')
    spec,spec_sdev=f_plot_spectrum_3d(sample,plot=False,spread_type='sdev')

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1


def f_compute_chisqr(dict_val,dict_sample,img_size):
    '''
    Compute chi-square values for sample w.r.t input images
    Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
    '''
    ### !!Both pixel histograms MUST have same bins and normalization!
    ### Compute chi-sqr
    ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
    ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
    chisqr_dict={}
    
    try: 
        val_dr=dict_val['hist_val'].copy()
        val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins

        sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2

        size=len(dict_val['hist_val'])
        l1,l2=int(size*0.3),int(size*0.7)
        keys=['chi_1a','chi_1b','chi_1c','chi_1']
        
        for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
            chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})

        idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
    #     chisqr_dict.update({'chi_sqr1':})

        chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],1.0))}) ## chi-sqr without denominator division
        chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data

        idx=img_size
        spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})

        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_sdev'][:idx]**2)})
        
        spec_loss=1.0*np.log(np.mean((dict_val['spec_val'][:idx]-dict_sample['spec_val'][:idx])**2))+1.0*np.log(np.mean((dict_val['spec_sdev'][:idx]-dict_sample['spec_sdev'][:idx])**2))
        chisqr_dict.update({'chi_spec3':spec_loss})
    
    except Exception as e: 
        print(e)
        
        keys=['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
        chisqr_dict=dict.fromkeys(keys,np.nan)
        pass
    
    return chisqr_dict
    

In [8]:
img_size=128
# img_size=128

In [9]:
val_data_dict={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing',
              '128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube'}

In [10]:
### Read validation data
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)

bkgnd_dict={}
num_bkgnd=10
sigma_lst=[0.5,0.65,0.8,1.1]
labels_lst=[0,1,2,3]
### Extract validation data   
for label in labels_lst:
    fname=val_data_dict[str(img_size)]+'/norm_1_sig_{0}_train_val.npy'.format(sigma_lst[label])
    print(fname)
    samples=np.load(fname,mmap_mode='r')[-num_bkgnd:][:,0,:,:,:]
    
    dict_val=f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(sigma_lst[label])]=dict_val
# del samples


/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.5_train_val.npy


<ipython-input-5-146a00d97e90>:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.65_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_1.1_train_val.npy


In [11]:
df_runs=pd.DataFrame([])

In [12]:
### bkgnd: Get bkgnd chi-squares for different sigma wrt one standard sigma 
lst=[0.5,0.65,0.8,1.1]
param_label=0.65
for count,run in enumerate(lst):
    num=10
    fname=val_data_dict[str(img_size)]+'/norm_1_sig_{0}_train_val.npy'.format(run)
    print(fname)    
    samples=np.load(fname,mmap_mode='r')[1000:1000+num][:,0,:,:,:]
    dict1={'label':'ref_%s'%(run),'fname':''}
    dict1['num_imgs']=num
    dict1['sigma']=run
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(samples,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample,img_size)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    
    df_runs=df_runs.append(dict1,ignore_index=True)


/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.5_train_val.npy


<ipython-input-5-146a00d97e90>:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.65_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_1.1_train_val.npy


### Add pytorch runs

In [13]:
## Pytorch code
### Load images

parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'


lst=[
        # ['20210628_73858_cgan_128_nodes1_lr0.0002_fm0_goodrun/',24970],
     # ['20210703_50233_cgan_128_nodes1_lr0.0002_vary_fm0_goodrun/',19360],
#      ['20210723_72050_cgan_128_nodes1_lr0.00002_finetune/',5070],
     ['20210723_74424_cgan_128_nodes1_lr0.000002_finetune/',1950],
     # ['20210726_173009_cgan_128_nodes1_lr0.000002_finetune/',3010],
    ]

# Prediction images 

for param_label in sigma_lst:
    for count,run in enumerate(lst):
        fldr,step=run[0],run[1]
#         print(count,run)
        prefix='run_%s_%s_%s'%(count,step,param_label)
        # prefix='best_%s'%(param_label)
        key=prefix
        main_dir=parent_dir+fldr
        print(key)
        fname=glob.glob(main_dir+'images/'+'inference_*label-{0}_*_step-{1}.npy'.format(param_label,step))[0]
        dict1={'label':key,'fname':fname}
        images=np.load(dict1['fname'])     ### No transform needed here

        images=images[:,0,:,:,:][:8] # Just take 8 images
        print(images.shape)
        print(np.max(images),np.min(images))

        dict1['num_imgs']=images.shape[0]
        dict1['sigma']=param_label
        print(dict1)
        ### Compute spectrum and histograms
        dict_sample=f_compute_hist_spect(images,bins)
        ### Compute chi squares
        dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample,img_size)

        dict1.update(dict_sample)
        dict1.update(dict_chisqrs)
        del(images)
        df_runs=df_runs.append(dict1,ignore_index=True)

run_0_1950_0.5
(8, 128, 128, 128)
0.99229246 -0.99592894
{'label': 'run_0_1950_0.5', 'fname': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210723_74424_cgan_128_nodes1_lr0.000002_finetune/images/inference_label-0.5_epoch-8_step-1950.npy', 'num_imgs': 8, 'sigma': 0.5}


<ipython-input-5-146a00d97e90>:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


run_0_1950_0.65
(8, 128, 128, 128)
0.99654293 -0.9974239
{'label': 'run_0_1950_0.65', 'fname': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210723_74424_cgan_128_nodes1_lr0.000002_finetune/images/inference_label-0.65_epoch-8_step-1950.npy', 'num_imgs': 8, 'sigma': 0.65}
run_0_1950_0.8
(8, 128, 128, 128)
0.9980289 -0.9984902
{'label': 'run_0_1950_0.8', 'fname': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210723_74424_cgan_128_nodes1_lr0.000002_finetune/images/inference_label-0.8_epoch-8_step-1950.npy', 'num_imgs': 8, 'sigma': 0.8}
run_0_1950_1.1
(8, 128, 128, 128)
0.99803746 -0.99935484
{'label': 'run_0_1950_1.1', 'fname': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210723_74424_cgan_128_nodes1_lr0.000002_finetune/images/inference_label-1.1_epoch-8_step-1950.npy', 'num_imgs': 8, 'sigma': 1.1}


In [14]:
df_runs.columns

Index(['chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2', 'chi_imgvar',
       'chi_spec1', 'chi_spec2', 'chi_spec3', 'fname', 'hist_bin_centers',
       'hist_err', 'hist_val', 'label', 'num_imgs', 'sigma', 'spec_sdev',
       'spec_val'],
      dtype='object')

In [15]:
col_list=['label','sigma', 'chi_1', 'chi_1a', 'chi_spec1', 'chi_spec3','chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2', 'hist_bin_centers', 'hist_err', 'hist_val','spec_sdev', 'spec_val','num_imgs','fname']

df_runs=df_runs.reindex(columns=col_list)


In [16]:
df_runs

,label,sigma,chi_1,chi_1a,chi_spec1,chi_spec3,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,hist_bin_centers,hist_err,hist_val,spec_sdev,spec_val,num_imgs,fname
0,ref_0.5,0.50,0.090948,0.061482,0.992893,56.332752,0.028066,0.001400,0.034883,0.651683,1144.471392,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.005936627991895166, 0.0027289643857619315, ...","[0.839080611545494, 0.5945387162404622, 0.0408...","[36711142.00061724, 9464870.801538132, 4309244...","[144876970.17498764, 83501521.84699954, 516112...",10.0,
1,ref_0.65,0.65,0.001031,0.000096,0.025629,55.449622,0.000518,0.000417,0.000033,1.170230,7.895298,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.009508233017504163, 0.004343966394681986, 0...","[1.0056309860061552, 0.5005160195533427, 0.044...","[46689915.0452183, 10823775.465715311, 4872186...","[179264555.21598023, 101328501.57421371, 59024...",10.0,
2,ref_0.8,0.80,0.050586,0.030094,0.387012,57.508217,0.018585,0.001907,0.019914,1.216073,597.527108,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.009317556425807215, 0.004323286526522787, 0...","[1.123868432559664, 0.43501240644783473, 0.044...","[44032598.67949965, 16082540.996525783, 706965...","[214863299.94639707, 111950934.26552913, 70320...",10.0,
3,ref_1.1,1.10,0.833944,0.177344,3.180629,62.953971,0.459085,0.197514,0.076790,2.412493,1578.689925,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.014308210603030396, 0.006402663975834263, 0...","[1.2376430453990972, 0.36783428488861364, 0.04...","[69795440.7508646, 17570686.601818357, 1955858...","[294000342.7371329, 162575123.2309894, 9523608...",10.0,
4,run_0_1950_0.5,0.50,0.020721,0.007864,17.678981,53.314291,0.007004,0.005853,0.005831,0.834341,22776.997726,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.007151546033683987, 0.003547488814532798, 0...","[0.8982560601902982, 0.5611502290394839, 0.043...","[30076299.08266418, 11635084.668773739, 344473...","[142516957.31490725, 86239271.82267304, 529290...",8.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
5,run_0_1950_0.65,0.65,0.024231,0.006107,17.902634,56.941664,0.003141,0.014983,0.003256,0.876579,13536.007971,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.007031743875138331, 0.003656502891031529, 0...","[1.0509949851462805, 0.4767332807038627, 0.043...","[42996267.24089591, 9329265.171115847, 5436485...","[158736140.38837218, 98489168.86987093, 602977...",8.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
6,run_0_1950_0.8,0.80,0.040562,0.006063,18.533948,58.337955,0.002770,0.031729,0.004779,0.602072,24402.511812,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.003990587203696556, 0.0019746544356346357, ...","[1.1854909907246531, 0.40095684284065947, 0.04...","[55788584.827463545, 13733958.05028585, 374354...","[192496186.99570775, 119765850.9637169, 674997...",8.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
7,run_0_1950_1.1,1.10,0.120887,0.006144,21.592614,58.020777,0.001131,0.113611,0.004188,0.733261,10545.457886,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.009337235209425282, 0.004254925380126485, 0...","[1.3325227517347187, 0.318610991326331, 0.0460...","[55376058.91049045, 10205385.70009818, 1024824...","[258606597.6869555, 150292123.73335904, 805397...",8.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...


### Compare chi-squares

In [17]:
chi_keys=['label','sigma','chi_1','chi_spec1', 'num_imgs', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','chi_spec3']
df_runs[chi_keys]

,label,sigma,chi_1,chi_spec1,num_imgs,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,chi_spec3
0,ref_0.5,0.50,0.090948,0.992893,10.0,0.061482,0.028066,0.001400,0.034883,0.651683,1144.471392,56.332752
1,ref_0.65,0.65,0.001031,0.025629,10.0,0.000096,0.000518,0.000417,0.000033,1.170230,7.895298,55.449622
2,ref_0.8,0.80,0.050586,0.387012,10.0,0.030094,0.018585,0.001907,0.019914,1.216073,597.527108,57.508217
3,ref_1.1,1.10,0.833944,3.180629,10.0,0.177344,0.459085,0.197514,0.076790,2.412493,1578.689925,62.953971
4,run_0_1950_0.5,0.50,0.020721,17.678981,8.0,0.007864,0.007004,0.005853,0.005831,0.834341,22776.997726,53.314291
5,run_0_1950_0.65,0.65,0.024231,17.902634,8.0,0.006107,0.003141,0.014983,0.003256,0.876579,13536.007971,56.941664
6,run_0_1950_0.8,0.80,0.040562,18.533948,8.0,0.006063,0.002770,0.031729,0.004779,0.602072,24402.511812,58.337955
7,run_0_1950_1.1,1.10,0.120887,21.592614,8.0,0.006144,0.001131,0.113611,0.004188,0.733261,10545.457886,58.020777


## Plot

In [18]:
def f_plot_hist_spec(df,ref_list,plot_type,img_size):

    assert plot_type in ['hist','spec','spec_relative','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y=row.spec_val
#             yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            yerr=row.spec_sdev
            x=np.arange(len(y))
            # y=x**2*y; yerr=x**2*yerr ## Plot k^2 P(y)
            plt.fill_between(x, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x, y, marker=marker, linestyle=':',label=label)

        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            print(row.sigma)
            dict_bkg=bkgnd_dict[str(row.sigma)]


            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row.num_imgs)
    
            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size/2)
            plt.legend()
            plt.title("relative spectrum")
        if plot_type=='grid':
            images=np.load(row.fname)[:,0,:,:,0]
            f_plot_grid(images[:8],cols=4,fig_size=(8,4))

            
    ### Plot input data
    for sig in ref_list: 
        dict_bkg=bkgnd_dict[str(sig)]
        if plot_type=='hist':
            x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
            x=f_invtransform(x)
            plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
            plt.title('Pixel Intensity Histogram')
            plt.xscale('symlog',linthreshx=50)

        if plot_type=='spec':
    #         y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
            y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']

            x=np.arange(len(y))
            # y=x**2*y; yerr=x**2*yerr ## Plot k^2 P(y)
            plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
            plt.title('Spectrum')
            plt.xlim(0,img_size)


        if plot_type=='spec_relative':
            plt.axhline(y=1.0,color='k',linestyle='-.')
            plt.title("relative spectrum")
            plt.xlim(0,img_size/2)
            plt.ylim(0.5,2)    

        if plot_type in ['hist','spec']:     
            plt.yscale('log')
        plt.legend(bbox_to_anchor=(0.5, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})

    
    plt.show()
# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_runs,dict_val,'spec_relative')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [19]:
df_runs.columns

Index(['label', 'sigma', 'chi_1', 'chi_1a', 'chi_spec1', 'chi_spec3', 'chi_1b',
       'chi_1c', 'chi_2', 'chi_imgvar', 'chi_spec2', 'hist_bin_centers',
       'hist_err', 'hist_val', 'spec_sdev', 'spec_val', 'num_imgs', 'fname'],
      dtype='object')

In [22]:
def f_widget_compare(df,labels_list,bkgnd,ref_list,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,ref_list,plot_type,img_size)

interact_manual(f_widget_compare,df=fixed(df_runs),
                ref_list=SelectMultiple(options=sigma_lst),
                labels_list=SelectMultiple(options=df_runs.label.values), 
                img_size=fixed(img_size),
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','spec_relative','grid']))

interactive(children=(SelectMultiple(description='labels_list', options=('ref_0.5', 'ref_0.65', 'ref_0.8', 're…

<function __main__.f_widget_compare(df, labels_list, bkgnd, ref_list, plot_type)>

In [21]:
col_list=['label','chi_1','chi_spec1','num_imgs','chi_2','chi_spec2']
df_runs[col_list]

,label,chi_1,chi_spec1,num_imgs,chi_2,chi_spec2
0,ref_0.5,0.090948,0.992893,10.0,0.034883,1144.471392
1,ref_0.65,0.001031,0.025629,10.0,0.000033,7.895298
2,ref_0.8,0.050586,0.387012,10.0,0.019914,597.527108
3,ref_1.1,0.833944,3.180629,10.0,0.076790,1578.689925
4,run_0_1950_0.5,0.020721,17.678981,8.0,0.005831,22776.997726
5,run_0_1950_0.65,0.024231,17.902634,8.0,0.003256,13536.007971
6,run_0_1950_0.8,0.040562,18.533948,8.0,0.004779,24402.511812
7,run_0_1950_1.1,0.120887,21.592614,8.0,0.004188,10545.457886
